In [ ]:
# Softmax classifier for guessing minesweeper board position and whether it has a mine or not

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt
import multiprocessing as mp

/home/ruben/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (8,8)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# Clears a square on the minesweeper board.
# If it had a mine, return true
# Otherwise if it has no adjacent mines, recursively run on adjacent squares
# Return false
def clearSquare(board,adjacency,row,col):
    rows,cols = dimensions
    if board[row,col] == 1:
        return True
    if adjacency[row,col] >= 0:
        return False
    n = 0
    for r in range(row-1,row+2):
        for c in range(col-1,col+2):
            if 0 <= r and r < rows and 0 <= c and c < cols:
                n += board[r,c]
    adjacency[row,col] = n
    if n == 0:
        for r in range(row-1,row+2):
            for c in range(col-1,col+2):
                if 0 <= r and r < rows and 0 <= c and c < cols:
                    clearSquare(board,adjacency,r,c)
    return False

In [4]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
# At least one square will be clear
def boardPartialMineCounts(board):
    clearProbability = r.uniform(0.05,0.5)
    result = np.full(dimensions,-1)
    didClear = False
    while not(didClear):
        for index, x in np.random.permutation(list(np.ndenumerate(board))):
            row,col = index
            if not(x) and result[row,col] == -1 and r.uniform(0,1) < clearProbability:
                clearSquare(board,result,row,col)
                didClear = True
    return result

In [5]:
# Generates a random training batch of size n
def randomBoard(i):
    return(np.random.random(dimensions) < mineProbability)

def encodeCountsOneHot(counts):
    countsOneHot = np.zeros((counts.size,10))
    countsOneHot[np.arange(counts.size), counts.flatten() + 1] = 1
    return(countsOneHot.flatten())

def validGuesses(boardAndCounts):
    board,counts = boardAndCounts
    validGuesses = np.append(((counts == -1).astype(int) - board).flatten().astype(float),
        board.flatten().astype(float))
    validGuessesSum = sum(validGuesses)
    if validGuessesSum > 0:
        return(validGuesses / validGuessesSum)
    else:
        return(np.zeros(board.size*2))

try:
    cpus = mp.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default

pool = mp.Pool(processes=cpus)

def next_training_batch(n):
    boards = pool.map(randomBoard, range(n))
    counts = pool.map(boardPartialMineCounts, boards)
    batch_xs = pool.map(encodeCountsOneHot, counts)
    batch_ys = pool.map(validGuesses, zip(boards,counts))
    return(batch_xs, batch_ys, boards)

Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-6:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/li

In [6]:
rows, cols = dimensions
size = rows*cols

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
model.add(tf.keras.layers.Dense(units=size, input_dim=size*10))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Reshape((rows, cols, 1)))

model.add(tf.keras.layers.Conv2D(32, (5, 5), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Reshape((4*4*32,)))

#model.add(tf.keras.layers.Dense(units=1024))
#model.add(tf.keras.layers.Activation('relu'))

#model.add(tf.keras.layers.Dropout(rate=0.5))

model.add(tf.keras.layers.Dense(units=size*2))
model.add(tf.keras.layers.Activation('softmax'))

In [ ]:
model.compile(optimizer='adam', loss='poisson')

In [ ]:
print(model.metrics_names)

In [7]:
savePath = "/media/ruben/BigDisk/tensorflow/tensorflow-logs/tf.Mines8"

In [9]:
model = tf.keras.models.load_model('{0}/model-3500.h5'.format(savePath))

In [10]:
for iteration in range(3501,1000001):
    x_batch, y_batch, _ = next_training_batch(10000)
    loss = model.train_on_batch(np.array(x_batch), np.array(y_batch))
    print('{0}: Loss at step {1}: {2}'.format(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    np.save('{0}/training_data/x_batch-{1}.npy'.format(savePath, iteration), np.array(x_batch))
    np.save('{0}/training_data/y_batch-{1}.npy'.format(savePath, iteration), np.array(y_batch))
    if iteration % 500 == 0:
        model.save('{0}/model-{1}.h5'.format(savePath, iteration))

2017-11-11 06:52:33: Loss at step 3501: 0.036921218037605286
2017-11-11 06:52:35: Loss at step 3502: 0.03676200285553932
2017-11-11 06:52:38: Loss at step 3503: 0.036942172795534134
2017-11-11 06:52:40: Loss at step 3504: 0.036985404789447784
2017-11-11 06:52:42: Loss at step 3505: 0.03677597641944885
2017-11-11 06:52:44: Loss at step 3506: 0.03680059313774109
2017-11-11 06:52:46: Loss at step 3507: 0.03687511757016182
2017-11-11 06:52:48: Loss at step 3508: 0.036910898983478546
2017-11-11 06:52:50: Loss at step 3509: 0.03695701062679291
2017-11-11 06:52:53: Loss at step 3510: 0.03688173368573189
2017-11-11 06:52:55: Loss at step 3511: 0.03668404370546341
2017-11-11 06:52:57: Loss at step 3512: 0.03682910278439522
2017-11-11 06:52:59: Loss at step 3513: 0.036751989275217056
2017-11-11 06:53:01: Loss at step 3514: 0.036820635199546814
2017-11-11 06:53:03: Loss at step 3515: 0.03678290545940399
2017-11-11 06:53:06: Loss at step 3516: 0.036781515926122665
2017-11-11 06:53:08: Loss at step

2017-11-11 06:57:27: Loss at step 3637: 0.036783184856176376
2017-11-11 06:57:30: Loss at step 3638: 0.036862097680568695
2017-11-11 06:57:32: Loss at step 3639: 0.03683163598179817
2017-11-11 06:57:34: Loss at step 3640: 0.03677985817193985
2017-11-11 06:57:36: Loss at step 3641: 0.03683692589402199
2017-11-11 06:57:38: Loss at step 3642: 0.03678060695528984
2017-11-11 06:57:41: Loss at step 3643: 0.036833830177783966
2017-11-11 06:57:43: Loss at step 3644: 0.036763835698366165
2017-11-11 06:57:45: Loss at step 3645: 0.03690661862492561
2017-11-11 06:57:47: Loss at step 3646: 0.03689483180642128
2017-11-11 06:57:49: Loss at step 3647: 0.03680440038442612
2017-11-11 06:57:51: Loss at step 3648: 0.03680308908224106
2017-11-11 06:57:53: Loss at step 3649: 0.03684573993086815
2017-11-11 06:57:55: Loss at step 3650: 0.03679755702614784
2017-11-11 06:57:57: Loss at step 3651: 0.036869700998067856
2017-11-11 06:58:00: Loss at step 3652: 0.036748021841049194
2017-11-11 06:58:02: Loss at step 

2017-11-11 07:02:22: Loss at step 3774: 0.03683708235621452
2017-11-11 07:02:25: Loss at step 3775: 0.03674326837062836
2017-11-11 07:02:27: Loss at step 3776: 0.036864910274744034
2017-11-11 07:02:29: Loss at step 3777: 0.03692103922367096
2017-11-11 07:02:31: Loss at step 3778: 0.03693343326449394
2017-11-11 07:02:33: Loss at step 3779: 0.03693638741970062
2017-11-11 07:02:35: Loss at step 3780: 0.03694935142993927
2017-11-11 07:02:38: Loss at step 3781: 0.036845654249191284
2017-11-11 07:02:40: Loss at step 3782: 0.03680649772286415
2017-11-11 07:02:42: Loss at step 3783: 0.036757029592990875
2017-11-11 07:02:44: Loss at step 3784: 0.036862291395664215
2017-11-11 07:02:46: Loss at step 3785: 0.036874134093523026
2017-11-11 07:02:48: Loss at step 3786: 0.03688378259539604
2017-11-11 07:02:51: Loss at step 3787: 0.036756839603185654
2017-11-11 07:02:53: Loss at step 3788: 0.036815494298934937
2017-11-11 07:02:55: Loss at step 3789: 0.036760784685611725
2017-11-11 07:02:57: Loss at ste

2017-11-11 07:07:13: Loss at step 3910: 0.03674496337771416
2017-11-11 07:07:15: Loss at step 3911: 0.0367145799100399
2017-11-11 07:07:17: Loss at step 3912: 0.03667007014155388
2017-11-11 07:07:20: Loss at step 3913: 0.03679288178682327
2017-11-11 07:07:22: Loss at step 3914: 0.03669046610593796
2017-11-11 07:07:24: Loss at step 3915: 0.03671008348464966
2017-11-11 07:07:26: Loss at step 3916: 0.03673507273197174
2017-11-11 07:07:28: Loss at step 3917: 0.03674417734146118
2017-11-11 07:07:30: Loss at step 3918: 0.03686463460326195
2017-11-11 07:07:32: Loss at step 3919: 0.036740049719810486
2017-11-11 07:07:35: Loss at step 3920: 0.036817535758018494
2017-11-11 07:07:37: Loss at step 3921: 0.03676178678870201
2017-11-11 07:07:39: Loss at step 3922: 0.03680400177836418
2017-11-11 07:07:41: Loss at step 3923: 0.03674845024943352
2017-11-11 07:07:43: Loss at step 3924: 0.03675954043865204
2017-11-11 07:07:45: Loss at step 3925: 0.03685895726084709
2017-11-11 07:07:48: Loss at step 3926:

2017-11-11 07:12:04: Loss at step 4046: 0.03684917092323303
2017-11-11 07:12:06: Loss at step 4047: 0.03681929409503937
2017-11-11 07:12:08: Loss at step 4048: 0.03674627095460892
2017-11-11 07:12:10: Loss at step 4049: 0.0369245745241642
2017-11-11 07:12:13: Loss at step 4050: 0.03674446418881416
2017-11-11 07:12:15: Loss at step 4051: 0.036795489490032196
2017-11-11 07:12:17: Loss at step 4052: 0.03675193339586258
2017-11-11 07:12:19: Loss at step 4053: 0.03674726560711861
2017-11-11 07:12:21: Loss at step 4054: 0.03670770302414894
2017-11-11 07:12:23: Loss at step 4055: 0.03677024692296982
2017-11-11 07:12:25: Loss at step 4056: 0.03679041564464569
2017-11-11 07:12:28: Loss at step 4057: 0.03675834834575653
2017-11-11 07:12:30: Loss at step 4058: 0.03678632155060768
2017-11-11 07:12:32: Loss at step 4059: 0.03678560629487038
2017-11-11 07:12:34: Loss at step 4060: 0.036765944212675095
2017-11-11 07:12:36: Loss at step 4061: 0.03680328279733658
2017-11-11 07:12:38: Loss at step 4062:

...

2017-11-11 07:56:14: Loss at step 5277: 0.03683433309197426
2017-11-11 07:56:16: Loss at step 5278: 0.03682652488350868
2017-11-11 07:56:18: Loss at step 5279: 0.03677448257803917
2017-11-11 07:56:20: Loss at step 5280: 0.03676701709628105
2017-11-11 07:56:22: Loss at step 5281: 0.03684748709201813
2017-11-11 07:56:25: Loss at step 5282: 0.03672650083899498
2017-11-11 07:56:27: Loss at step 5283: 0.03678348660469055
2017-11-11 07:56:29: Loss at step 5284: 0.03675515577197075
2017-11-11 07:56:31: Loss at step 5285: 0.03686431050300598
2017-11-11 07:56:33: Loss at step 5286: 0.03676925227046013
2017-11-11 07:56:35: Loss at step 5287: 0.03671221435070038
2017-11-11 07:56:38: Loss at step 5288: 0.03674933314323425
2017-11-11 07:56:40: Loss at step 5289: 0.036709416657686234
2017-11-11 07:56:42: Loss at step 5290: 0.036714378744363785
2017-11-11 07:56:44: Loss at step 5291: 0.03668693080544472
2017-11-11 07:56:46: Loss at step 5292: 0.03683823347091675
2017-11-11 07:56:48: Loss at step 5293

2017-11-11 08:01:07: Loss at step 5414: 0.03678181767463684
2017-11-11 08:01:10: Loss at step 5415: 0.03680938854813576
2017-11-11 08:01:12: Loss at step 5416: 0.03675444796681404
2017-11-11 08:01:14: Loss at step 5417: 0.03679803013801575
2017-11-11 08:01:16: Loss at step 5418: 0.03680841624736786
2017-11-11 08:01:18: Loss at step 5419: 0.03678342327475548
2017-11-11 08:01:21: Loss at step 5420: 0.0367349237203598
2017-11-11 08:01:23: Loss at step 5421: 0.036759618669748306
2017-11-11 08:01:25: Loss at step 5422: 0.036769043654203415
2017-11-11 08:01:27: Loss at step 5423: 0.03674799203872681
2017-11-11 08:01:29: Loss at step 5424: 0.03675438091158867
2017-11-11 08:01:32: Loss at step 5425: 0.03683612868189812
2017-11-11 08:01:34: Loss at step 5426: 0.036764197051525116
2017-11-11 08:01:36: Loss at step 5427: 0.03680302947759628
2017-11-11 08:01:38: Loss at step 5428: 0.036851417273283005
2017-11-11 08:01:40: Loss at step 5429: 0.03684091195464134
2017-11-11 08:01:43: Loss at step 543

2017-11-11 08:06:03: Loss at step 5551: 0.036704663187265396
2017-11-11 08:06:05: Loss at step 5552: 0.036715950816869736
2017-11-11 08:06:07: Loss at step 5553: 0.036708857864141464
2017-11-11 08:06:09: Loss at step 5554: 0.03671044111251831
2017-11-11 08:06:11: Loss at step 5555: 0.03673554211854935
2017-11-11 08:06:13: Loss at step 5556: 0.03674839437007904
2017-11-11 08:06:16: Loss at step 5557: 0.0367605984210968
2017-11-11 08:06:18: Loss at step 5558: 0.036808136850595474
2017-11-11 08:06:20: Loss at step 5559: 0.036758940666913986
2017-11-11 08:06:22: Loss at step 5560: 0.03677215427160263
2017-11-11 08:06:24: Loss at step 5561: 0.036749131977558136
2017-11-11 08:06:26: Loss at step 5562: 0.036733854562044144
2017-11-11 08:06:29: Loss at step 5563: 0.03674313426017761
2017-11-11 08:06:31: Loss at step 5564: 0.03674943372607231
2017-11-11 08:06:33: Loss at step 5565: 0.03677045926451683
2017-11-11 08:06:35: Loss at step 5566: 0.036753296852111816
2017-11-11 08:06:37: Loss at step

2017-11-11 08:11:00: Loss at step 5688: 0.03686915710568428


KeyboardInterrupt: 

In [16]:
def load_training_batch(n):
    x_batch = np.load('{0}/training_data/x_batch-{1}.npy'.format(savePath, n))
    y_batch = np.load('{0}/training_data/y_batch-{1}.npy'.format(savePath, n))
    return (x_batch, y_batch)

In [ ]:
for iteration in range(5689,1000001):
    x_batch, y_batch = load_training_batch(iteration % 5689)
    loss = model.train_on_batch(np.array(x_batch), np.array(y_batch))
    print('{0}: Loss at step {1}: {2}'.format(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    if iteration % 500 == 0:
        model.save('{0}/model-{1}.h5'.format(savePath, iteration))

2017-11-11 08:13:21: Loss at step 5689: 0.03686288744211197
2017-11-11 08:13:22: Loss at step 5690: 0.036672044545412064
2017-11-11 08:13:23: Loss at step 5691: 0.0370478555560112
2017-11-11 08:13:23: Loss at step 5692: 0.03694919869303703
2017-11-11 08:13:24: Loss at step 5693: 0.03681319206953049
2017-11-11 08:13:24: Loss at step 5694: 0.03692269325256348
2017-11-11 08:13:25: Loss at step 5695: 0.03690475970506668
2017-11-11 08:13:25: Loss at step 5696: 0.03696698695421219
2017-11-11 08:13:26: Loss at step 5697: 0.03687051311135292
2017-11-11 08:13:26: Loss at step 5698: 0.03680562227964401
2017-11-11 08:13:27: Loss at step 5699: 0.03691377490758896
2017-11-11 08:13:27: Loss at step 5700: 0.03687459975481033
2017-11-11 08:13:28: Loss at step 5701: 0.03696953132748604
2017-11-11 08:13:28: Loss at step 5702: 0.0369834341108799
2017-11-11 08:13:29: Loss at step 5703: 0.036900438368320465
2017-11-11 08:13:29: Loss at step 5704: 0.0368473194539547
2017-11-11 08:13:30: Loss at step 5705: 0

In [80]:
# Run a test
batchSize = 70000
x_batch, y_batch, _ = next_training_batch(batchSize)

predictions = model.predict(np.array(x_batch), batch_size = batchSize)
bestSquares = [pred.argmax() for pred in predictions]
unfrees = [(ys == 0).astype(int) for ys in y_batch]
frees = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(frees))

Number of errors for batch size of  70000
9352


In [81]:
900*4

3600

In [82]:
900*7

6300